<a href="https://colab.research.google.com/github/haticetheengineer/Customer_Segmentation_FLO_CRM/blob/main/FLO_CRM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pwd

/content


In [ ]:
cd /content/drive/MyDrive/miuul_data_science_bootcamp/homeworks/CRM_Analytics/FLOMusteriSegmentasyonu/

/content/drive/MyDrive/miuul_data_science_bootcamp/homeworks/CRM_Analytics/FLOMusteriSegmentasyonu


In [ ]:
!ls

FLO_CRM.ipynb  flo_data_20k.csv  FLO_RFM_Analizi.pdf  FLO_RFM.py



#  Customer Segmentation with RFM

**Business Problem**

* FLO wants to segment its customers and develop marketing strategies based on these segments.
* To achieve this, customer behaviors will be defined and groups will be created based on these behavior clusters.


# Dataset Story

 The dataset consists of information obtained from the past shopping behaviors of customers who made their last purchases between 2020 and 2021 as OmniChannel (shopping both online and offline).

 * master_id: Unique customer number
 * order_channel: The channel used for shopping on the platform (Android, iOS, Desktop, Mobile, Offline)
 * last_order_channel: The channel used for the most recent purchase
 * first_order_date: The date of the customer's first purchase
 * last_order_date: The date of the customer's most recent purchase
 * last_order_date_online: The date of the customer's most recent online purchase
 * last_order_date_offline: The date of the customer's most recent offline purchase
 * order_num_total_ever_online: The total number of purchases made by the customer on the online platform
 * order_num_total_ever_offline: The total number of purchases made by the customer offline
 * customer_value_total_ever_offline: The total amount spent by the customer in offline purchases
 * customer_value_total_ever_online: The total amount spent by the customer in online purchases
 * interested_in_categories_12: List of categories the customer has shopped in over the past 12 months



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

# TASK 1: Data Understanding and Preparation
* **Step 1:** Read the `flo_data_20K.csv` dataset.
* **Step 2:** In the dataset:
   * a. Examine the first 10 observations,
   * b. Check the variable names,
   * c. Review descriptive statistics,
   * d. Check for missing values,
   * e. Review variable types.
* **Step 3:** Omnichannel customers shop on both online and offline platforms. Create new variables for each customer representing the total number of purchases and total spending.
* **Step 4:** Review the data types of variables. Convert date-related variables to `date` type.
* **Step 5:** Analyze the distribution of the number of customers in each shopping channel, the average number of products purchased, and average spending.
* **Step 6:** List the top 10 customers with the highest revenue.
* **Step 7:** List the top 10 customers with the highest number of orders.
* **Step 8:** Functionize the data preparation process.


In [ ]:
df_ = pd.read_csv('flo_data_20k.csv')
df = df_.copy()
print(df.head(10))

                              master_id order_channel last_order_channel  \
0  cc294636-19f0-11eb-8d74-000d3a38a36f   Android App            Offline   
1  f431bd5a-ab7b-11e9-a2fc-000d3a38a36f   Android App             Mobile   
2  69b69676-1a40-11ea-941b-000d3a38a36f   Android App        Android App   
3  1854e56c-491f-11eb-806e-000d3a38a36f   Android App        Android App   
4  d6ea1074-f1f5-11e9-9346-000d3a38a36f       Desktop            Desktop   
5  e585280e-aae1-11e9-a2fc-000d3a38a36f       Desktop            Offline   
6  c445e4ee-6242-11ea-9d1a-000d3a38a36f   Android App        Android App   
7  3f1b4dc8-8a7d-11ea-8ec0-000d3a38a36f        Mobile            Offline   
8  cfbda69e-5b4f-11ea-aca7-000d3a38a36f   Android App        Android App   
9  1143f032-440d-11ea-8b43-000d3a38a36f        Mobile             Mobile   

  first_order_date last_order_date last_order_date_online  \
0       2020-10-30      2021-02-26             2021-02-21   
1       2017-02-08      2021-02-16       

In [ ]:
df.columns

Index(['master_id', 'order_channel', 'last_order_channel', 'first_order_date',
       'last_order_date', 'last_order_date_online', 'last_order_date_offline',
       'order_num_total_ever_online', 'order_num_total_ever_offline',
       'customer_value_total_ever_offline', 'customer_value_total_ever_online',
       'interested_in_categories_12'],
      dtype='object')

In [ ]:
df.nunique()

,0
master_id,19945
order_channel,4
last_order_channel,5
first_order_date,2465
last_order_date,366
last_order_date_online,1743
last_order_date_offline,738
order_num_total_ever_online,57
order_num_total_ever_offline,32
customer_value_total_ever_offline,6097


In [ ]:
df.dtypes

,0
master_id,object
order_channel,object
last_order_channel,object
first_order_date,datetime64[ns]
last_order_date,datetime64[ns]
last_order_date_online,datetime64[ns]
last_order_date_offline,datetime64[ns]
order_num_total_ever_online,float64
order_num_total_ever_offline,float64
customer_value_total_ever_offline,float64


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   master_id                          19945 non-null  object 
 1   order_channel                      19945 non-null  object 
 2   last_order_channel                 19945 non-null  object 
 3   first_order_date                   19945 non-null  object 
 4   last_order_date                    19945 non-null  object 
 5   last_order_date_online             19945 non-null  object 
 6   last_order_date_offline            19945 non-null  object 
 7   order_num_total_ever_online        19945 non-null  float64
 8   order_num_total_ever_offline       19945 non-null  float64
 9   customer_value_total_ever_offline  19945 non-null  float64
 10  customer_value_total_ever_online   19945 non-null  float64
 11  interested_in_categories_12        19945 non-null  obj

In [ ]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
order_num_total_ever_online,19945.00000,3.11085,4.22565,1.00000,1.00000,2.00000,4.00000,200.00000
order_num_total_ever_offline,19945.00000,1.91391,2.06288,1.00000,1.00000,1.00000,2.00000,109.00000
customer_value_total_ever_offline,19945.00000,253.92260,301.53285,10.00000,99.99000,179.98000,319.97000,18119.14000
customer_value_total_ever_online,19945.00000,497.32169,832.60189,12.99000,149.98000,286.46000,578.44000,45220.13000


In [ ]:
df.shape

(19945, 12)

In [ ]:
df.isnull().sum()

,0
master_id,0
order_channel,0
last_order_channel,0
first_order_date,0
last_order_date,0
last_order_date_online,0
last_order_date_offline,0
order_num_total_ever_online,0
order_num_total_ever_offline,0
customer_value_total_ever_offline,0


In [ ]:
# Step 3: Omnichannel customers shop on both online and offline platforms.
# Create new variables for each customer to represent the total number of purchases and total spending.

df['sum_of_orders']= df['order_num_total_ever_online'] + df['order_num_total_ever_offline']
df['sum_of_evers']= df['customer_value_total_ever_offline'] + df['customer_value_total_ever_online']
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,sum_of_orders,sum_of_evers
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.00000,1.00000,139.99000,799.38000,[KADIN],5.00000,939.37000
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.00000,2.00000,159.97000,1853.58000,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",21.00000,2013.55000
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.00000,2.00000,189.97000,395.35000,"[ERKEK, KADIN]",5.00000,585.32000
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.00000,1.00000,39.99000,81.98000,"[AKTIFCOCUK, COCUK]",2.00000,121.97000
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.00000,1.00000,49.99000,159.99000,[AKTIFSPOR],2.00000,209.98000


In [ ]:
# Step 4: Review the variable types. Convert date-related variables to `date` type.
df.columns.str.contains('date')

array([False, False, False,  True,  True,  True,  True, False, False,
       False, False, False, False, False])

In [ ]:
df.columns[df.columns.str.contains('date')]

Index(['first_order_date', 'last_order_date', 'last_order_date_online',
       'last_order_date_offline'],
      dtype='object')

In [ ]:
date_columns = df.columns[df.columns.str.contains('date')]
date_columns

Index(['first_order_date', 'last_order_date', 'last_order_date_online',
       'last_order_date_offline'],
      dtype='object')

In [ ]:
df[date_columns] = df[date_columns].apply(pd.to_datetime)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   master_id                          19945 non-null  object        
 1   order_channel                      19945 non-null  object        
 2   last_order_channel                 19945 non-null  object        
 3   first_order_date                   19945 non-null  datetime64[ns]
 4   last_order_date                    19945 non-null  datetime64[ns]
 5   last_order_date_online             19945 non-null  datetime64[ns]
 6   last_order_date_offline            19945 non-null  datetime64[ns]
 7   order_num_total_ever_online        19945 non-null  float64       
 8   order_num_total_ever_offline       19945 non-null  float64       
 9   customer_value_total_ever_offline  19945 non-null  float64       
 10  customer_value_total_ever_online  

In [ ]:
# Step 5: Analyze the distribution of the number of customers in each shopping channel,
# the average number of products purchased, and average spending.

df.groupby('order_channel').agg({'master_id': 'count',
                                 'sum_of_orders': 'mean',
                                 'sum_of_evers': 'mean'})

,master_id,sum_of_orders,sum_of_evers
order_channel,,,
Android App,9495,5.50490,823.49266
Desktop,2735,3.99269,588.78298
Ios App,2833,5.41864,891.63429
Mobile,4882,4.44060,620.27512


In [ ]:
df['order_channel'].unique()

array(['Android App', 'Desktop', 'Mobile', 'Ios App'], dtype=object)

In [ ]:
# Step 6: List the top 10 customers with the highest revenue.
df.sort_values('sum_of_evers', ascending = False).head(10)

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,sum_of_orders,sum_of_evers
11150,5d1c466a-9cfd-11e9-9897-000d3a38a36f,Android App,Desktop,2013-10-11,2021-04-30,2021-04-30,2020-12-24,200.00000,2.00000,684.97000,45220.13000,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",202.00000,45905.10000
4315,d5ef8058-a5c6-11e9-a2fc-000d3a38a36f,Android App,Android App,2018-08-06,2021-02-23,2021-02-23,2020-07-06,67.00000,1.00000,130.49000,36687.80000,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",68.00000,36818.29000
7613,73fd19aa-9e37-11e9-9897-000d3a38a36f,Ios App,Offline,2014-01-14,2021-05-18,2021-01-30,2021-05-18,81.00000,1.00000,1263.76000,32654.34000,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",82.00000,33918.10000
13880,7137a5c0-7aad-11ea-8f20-000d3a38a36f,Ios App,Offline,2021-03-01,2021-04-13,2021-03-18,2021-04-13,10.00000,1.00000,538.94000,30688.47000,"[ERKEK, KADIN, AKTIFSPOR]",11.00000,31227.41000
9055,47a642fe-975b-11eb-8c2a-000d3a38a36f,Android App,Offline,2021-04-07,2021-04-27,2021-04-07,2021-04-27,1.00000,3.00000,18119.14000,2587.20000,[AKTIFSPOR],4.00000,20706.34000
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,Desktop,Desktop,2020-02-16,2021-04-30,2021-04-30,2020-12-18,66.00000,4.00000,843.68000,17599.89000,"[ERKEK, KADIN, AKTIFSPOR]",70.00000,18443.57000
8068,d696c654-2633-11ea-8e1c-000d3a38a36f,Ios App,Ios App,2017-05-10,2021-04-13,2021-04-13,2019-08-15,69.00000,1.00000,82.48000,16836.09000,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",70.00000,16918.57000
163,fef57ffa-aae6-11e9-a2fc-000d3a38a36f,Mobile,Desktop,2016-11-08,2021-05-12,2021-05-12,2020-07-09,36.00000,1.00000,180.73000,12545.37000,"[ERKEK, AKTIFSPOR]",37.00000,12726.10000
7223,cba59206-9dd1-11e9-9897-000d3a38a36f,Android App,Android App,2013-02-21,2021-05-09,2021-05-09,2020-01-25,130.00000,1.00000,49.99000,12232.25000,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",131.00000,12282.24000
18767,fc0ce7a4-9d87-11e9-9897-000d3a38a36f,Desktop,Desktop,2018-11-24,2020-11-11,2020-11-11,2019-12-06,18.00000,2.00000,64.97000,12038.18000,"[ERKEK, KADIN]",20.00000,12103.15000


In [ ]:
df.groupby('master_id').agg({'sum_of_evers':'sum'}).sort_values('sum_of_evers', ascending = False).head(10)

,sum_of_evers
master_id,
5d1c466a-9cfd-11e9-9897-000d3a38a36f,45905.10000
d5ef8058-a5c6-11e9-a2fc-000d3a38a36f,36818.29000
73fd19aa-9e37-11e9-9897-000d3a38a36f,33918.10000
7137a5c0-7aad-11ea-8f20-000d3a38a36f,31227.41000
47a642fe-975b-11eb-8c2a-000d3a38a36f,20706.34000
a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,18443.57000
d696c654-2633-11ea-8e1c-000d3a38a36f,16918.57000
fef57ffa-aae6-11e9-a2fc-000d3a38a36f,12726.10000
cba59206-9dd1-11e9-9897-000d3a38a36f,12282.24000


In [ ]:
# Step 7: List the top 10 customers with the highest number of orders.
df[['master_id', 'sum_of_orders']].sort_values('sum_of_orders', ascending = False).head(10)

,master_id,sum_of_orders
11150,5d1c466a-9cfd-11e9-9897-000d3a38a36f,202.00000
7223,cba59206-9dd1-11e9-9897-000d3a38a36f,131.00000
8783,a57f4302-b1a8-11e9-89fa-000d3a38a36f,111.00000
2619,fdbe8304-a7ab-11e9-a2fc-000d3a38a36f,88.00000
6322,329968c6-a0e2-11e9-a2fc-000d3a38a36f,83.00000
7613,73fd19aa-9e37-11e9-9897-000d3a38a36f,82.00000
9347,44d032ee-a0d4-11e9-a2fc-000d3a38a36f,77.00000
10954,b27e241a-a901-11e9-a2fc-000d3a38a36f,75.00000
8068,d696c654-2633-11ea-8e1c-000d3a38a36f,70.00000
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,70.00000


In [ ]:
df.groupby('master_id').agg({'sum_of_orders':'sum'}).sort_values('sum_of_orders', ascending = False).head(10)


,sum_of_orders
master_id,
5d1c466a-9cfd-11e9-9897-000d3a38a36f,202.00000
cba59206-9dd1-11e9-9897-000d3a38a36f,131.00000
a57f4302-b1a8-11e9-89fa-000d3a38a36f,111.00000
fdbe8304-a7ab-11e9-a2fc-000d3a38a36f,88.00000
329968c6-a0e2-11e9-a2fc-000d3a38a36f,83.00000
73fd19aa-9e37-11e9-9897-000d3a38a36f,82.00000
44d032ee-a0d4-11e9-a2fc-000d3a38a36f,77.00000
b27e241a-a901-11e9-a2fc-000d3a38a36f,75.00000
d696c654-2633-11ea-8e1c-000d3a38a36f,70.00000


In [ ]:
df.sort_values('sum_of_orders', ascending = False).head(10)

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,sum_of_orders,sum_of_evers
11150,5d1c466a-9cfd-11e9-9897-000d3a38a36f,Android App,Desktop,2013-10-11,2021-04-30,2021-04-30,2020-12-24,200.00000,2.00000,684.97000,45220.13000,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",202.00000,45905.10000
7223,cba59206-9dd1-11e9-9897-000d3a38a36f,Android App,Android App,2013-02-21,2021-05-09,2021-05-09,2020-01-25,130.00000,1.00000,49.99000,12232.25000,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",131.00000,12282.24000
8783,a57f4302-b1a8-11e9-89fa-000d3a38a36f,Android App,Offline,2019-08-07,2020-11-04,2020-09-07,2020-11-04,2.00000,109.00000,10239.46000,143.98000,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",111.00000,10383.44000
2619,fdbe8304-a7ab-11e9-a2fc-000d3a38a36f,Android App,Offline,2018-10-18,2020-06-30,2018-10-18,2020-06-30,1.00000,87.00000,8432.25000,139.98000,[ERKEK],88.00000,8572.23000
6322,329968c6-a0e2-11e9-a2fc-000d3a38a36f,Ios App,Ios App,2019-02-14,2021-04-05,2021-04-05,2020-02-17,2.00000,81.00000,3997.55000,242.81000,[ERKEK],83.00000,4240.36000
7613,73fd19aa-9e37-11e9-9897-000d3a38a36f,Ios App,Offline,2014-01-14,2021-05-18,2021-01-30,2021-05-18,81.00000,1.00000,1263.76000,32654.34000,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",82.00000,33918.10000
9347,44d032ee-a0d4-11e9-a2fc-000d3a38a36f,Mobile,Mobile,2019-02-11,2021-02-11,2021-02-11,2020-12-24,3.00000,74.00000,4640.77000,543.28000,"[KADIN, AKTIFSPOR]",77.00000,5184.05000
10954,b27e241a-a901-11e9-a2fc-000d3a38a36f,Mobile,Mobile,2015-09-12,2021-04-01,2021-04-01,2019-10-08,72.00000,3.00000,292.93000,5004.95000,"[AKTIFCOCUK, ERKEK, KADIN, AKTIFSPOR]",75.00000,5297.88000
8068,d696c654-2633-11ea-8e1c-000d3a38a36f,Ios App,Ios App,2017-05-10,2021-04-13,2021-04-13,2019-08-15,69.00000,1.00000,82.48000,16836.09000,"[AKTIFCOCUK, ERKEK, COCUK, KADIN, AKTIFSPOR]",70.00000,16918.57000
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,Desktop,Desktop,2020-02-16,2021-04-30,2021-04-30,2020-12-18,66.00000,4.00000,843.68000,17599.89000,"[ERKEK, KADIN, AKTIFSPOR]",70.00000,18443.57000


In [ ]:
# Step 8: Functionize the data preparation process.
def data_prep(dataframe):
  dataframe['sum_of_orders'] = dataframe['order_num_total_ever_online'] + dataframe['order_num_total_ever_offline']
  dataframe['sum_of_evers'] = dataframe['customer_value_total_ever_offline'] + dataframe['customer_value_total_ever_online']

  date_columns = dataframe.columns[dataframe.columns.str.contains('date')]
  dataframe[date_columns] = dataframe[date_columns].apply(pd.to_datetime)

  return dataframe


# TASK 2: Calculating RFM Metrics


In [ ]:
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,sum_of_orders,sum_of_evers
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.00000,1.00000,139.99000,799.38000,[KADIN],5.00000,939.37000
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.00000,2.00000,159.97000,1853.58000,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",21.00000,2013.55000
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.00000,2.00000,189.97000,395.35000,"[ERKEK, KADIN]",5.00000,585.32000
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.00000,1.00000,39.99000,81.98000,"[AKTIFCOCUK, COCUK]",2.00000,121.97000
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.00000,1.00000,49.99000,159.99000,[AKTIFSPOR],2.00000,209.98000


In [ ]:
df["last_order_date"].max()

Timestamp('2021-05-30 00:00:00')

*Note that: To calculate the recency value, you can choose the analysis date as 2 days after the maximum date.*


In [ ]:
today_date = dt.datetime(2021, 6, 1)
today_date

datetime.datetime(2021, 6, 1, 0, 0)

In [ ]:
type(today_date)

datetime.datetime

In [ ]:
rfm = df.groupby('master_id').agg({'last_order_date': lambda last_order_date: (today_date - last_order_date.max()).days,
                                   'sum_of_orders': lambda sum_of_orders: sum_of_orders.sum(),
                                   'sum_of_evers': lambda sum_of_evers: sum_of_evers.sum()})
rfm.head(10)

,last_order_date,sum_of_orders,sum_of_evers
master_id,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,10,5.00000,776.07000
00034aaa-a838-11e9-a2fc-000d3a38a36f,298,3.00000,269.47000
000be838-85df-11ea-a90b-000d3a38a36f,213,4.00000,722.69000
000c1fe2-a8b7-11ea-8479-000d3a38a36f,27,7.00000,874.16000
000f5e3e-9dde-11ea-80cd-000d3a38a36f,20,7.00000,1620.33000
00136ce2-a562-11e9-a2fc-000d3a38a36f,203,2.00000,359.45000
00142f9a-7af6-11eb-8460-000d3a38a36f,25,3.00000,404.94000
0014778a-5b11-11ea-9a2c-000d3a38a36f,26,3.00000,727.43000
0018c6aa-ab6c-11e9-a2fc-000d3a38a36f,126,2.00000,317.91000


In [ ]:
rfm.columns = ['recency', 'frequency', 'monetary']
rfm.head()

,recency,frequency,monetary
master_id,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,10,5.00000,776.07000
00034aaa-a838-11e9-a2fc-000d3a38a36f,298,3.00000,269.47000
000be838-85df-11ea-a90b-000d3a38a36f,213,4.00000,722.69000
000c1fe2-a8b7-11ea-8479-000d3a38a36f,27,7.00000,874.16000
000f5e3e-9dde-11ea-80cd-000d3a38a36f,20,7.00000,1620.33000


In [ ]:
df[df.master_id.str.contains("00016786-2f5a-11ea-bb80-000d3a38a36f")]

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,sum_of_orders,sum_of_evers
17817,00016786-2f5a-11ea-bb80-000d3a38a36f,Mobile,Mobile,2019-11-19,2021-05-22,2021-05-22,2020-10-05,2.00000,3.00000,427.19000,348.88000,"[ERKEK, COCUK, AKTIFSPOR]",5.00000,776.07000


In [ ]:
df[df.master_id.str.contains("cc294636-19f0-11eb-8d74-000d3a38a36f")]

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,sum_of_orders,sum_of_evers
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.00000,1.00000,139.99000,799.38000,[KADIN],5.00000,939.37000


In [ ]:
rfm[rfm.index.str.contains("cc294636-19f0-11eb-8d74-000d3a38a36f")]

,recency,frequency,monetary
master_id,,,
cc294636-19f0-11eb-8d74-000d3a38a36f,95,5.00000,939.37000


In [ ]:
rfm.shape

(19945, 3)

# **Task 3: Calculating RF Scores**

* **Step 1:** Convert the Recency, Frequency, and Monetary metrics into scores ranging from 1 to 5 using `qcut`.
* **Step 2:** Save these scores as `recency_score`, `frequency_score`, and `monetary_score`.
* **Step 3:** Combine `recency_score` and `frequency_score` into a single variable and save it as `RF_SCORE`.




In [ ]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,19945.00000,134.45836,103.28115,2.00000,43.00000,111.00000,202.00000,367.00000
frequency,19945.00000,5.02477,4.74271,2.00000,3.00000,4.00000,6.00000,202.00000
monetary,19945.00000,751.24429,895.40217,44.98000,339.98000,545.27000,897.78000,45905.10000


In [ ]:
rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])

# 0-100, 0-20, 20-40, 40-60, 60-80, 80-100

rfm["frequency_score"] = pd.qcut(rfm['frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])

rfm["monetary_score"] = pd.qcut(rfm['monetary'], 5, labels=[1, 2, 3, 4, 5])

rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score
master_id,,,,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,10,5.00000,776.07000,5,4,4
00034aaa-a838-11e9-a2fc-000d3a38a36f,298,3.00000,269.47000,1,2,1
000be838-85df-11ea-a90b-000d3a38a36f,213,4.00000,722.69000,2,3,4
000c1fe2-a8b7-11ea-8479-000d3a38a36f,27,7.00000,874.16000,5,4,4
000f5e3e-9dde-11ea-80cd-000d3a38a36f,20,7.00000,1620.33000,5,4,5


In [ ]:
rfm["RF_SCORE"] = (rfm['recency_score'].astype(str) +
                    rfm['frequency_score'].astype(str))
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE
master_id,,,,,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,10,5.00000,776.07000,5,4,4,54
00034aaa-a838-11e9-a2fc-000d3a38a36f,298,3.00000,269.47000,1,2,1,12
000be838-85df-11ea-a90b-000d3a38a36f,213,4.00000,722.69000,2,3,4,23
000c1fe2-a8b7-11ea-8479-000d3a38a36f,27,7.00000,874.16000,5,4,4,54
000f5e3e-9dde-11ea-80cd-000d3a38a36f,20,7.00000,1620.33000,5,4,5,54


In [ ]:
rfm["RFM_SCORE"] = (rfm['recency_score'].astype(str) + rfm['frequency_score'].astype(str) + rfm['monetary_score'].astype(str))

rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE,RFM_SCORE
master_id,,,,,,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,10,5.00000,776.07000,5,4,4,54,544
00034aaa-a838-11e9-a2fc-000d3a38a36f,298,3.00000,269.47000,1,2,1,12,121
000be838-85df-11ea-a90b-000d3a38a36f,213,4.00000,722.69000,2,3,4,23,234
000c1fe2-a8b7-11ea-8479-000d3a38a36f,27,7.00000,874.16000,5,4,4,54,544
000f5e3e-9dde-11ea-80cd-000d3a38a36f,20,7.00000,1620.33000,5,4,5,54,545


In [ ]:
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,19945.00000,134.45836,103.28115,2.00000,43.00000,111.00000,202.00000,367.00000
frequency,19945.00000,5.02477,4.74271,2.00000,3.00000,4.00000,6.00000,202.00000
monetary,19945.00000,751.24429,895.40217,44.98000,339.98000,545.27000,897.78000,45905.10000


In [ ]:
rfm[rfm["RF_SCORE"] == "55"]

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE,RFM_SCORE
master_id,,,,,,,,
004d5204-2037-11ea-87bf-000d3a38a36f,27,8.00000,1170.76000,5,5,5,55,555
00736820-a834-11e9-a2fc-000d3a38a36f,26,9.00000,714.53000,5,5,4,55,554
00b3ee24-aa44-11e9-a2fc-000d3a38a36f,24,8.00000,2027.78000,5,5,5,55,555
00cf8494-9da2-11e9-9897-000d3a38a36f,5,53.00000,6275.33000,5,5,5,55,555
0151bbee-a7de-11e9-a2fc-000d3a38a36f,14,18.00000,2649.02000,5,5,5,55,555
...,...,...,...,...,...,...,...,...
fe834606-c9d0-11ea-a31e-000d3a38a36f,18,9.00000,2825.10000,5,5,5,55,555
fee58510-5b48-11ea-b47b-000d3a38a36f,11,9.00000,991.44000,5,5,4,55,554
fef57ffa-aae6-11e9-a2fc-000d3a38a36f,20,37.00000,12726.10000,5,5,5,55,555


In [ ]:
rfm[rfm["RF_SCORE"] == "11"]

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE,RFM_SCORE
master_id,,,,,,,,
0033f078-7359-11ea-92d0-000d3a38a36f,360,2.00000,153.98000,1,1,1,11,111
005c7914-9e78-11e9-9897-000d3a38a36f,279,2.00000,264.48000,1,1,1,11,111
0082e79e-5b50-11ea-ba9b-000d3a38a36f,318,2.00000,279.97000,1,1,1,11,111
00ede098-9e6e-11e9-9897-000d3a38a36f,332,2.00000,339.99000,1,1,2,11,112
010a7a9c-94d0-11ea-85df-000d3a38a36f,314,2.00000,485.32000,1,1,3,11,113
...,...,...,...,...,...,...,...,...
d826e8de-aa71-11e9-a2fc-000d3a38a36f,312,2.00000,359.97000,1,1,2,11,112
d850c728-8d81-11ea-8f41-000d3a38a36f,314,2.00000,364.91000,1,1,2,11,112
d8624464-c4e8-11ea-98f7-000d3a38a36f,315,2.00000,629.91000,1,1,3,11,113


# **Task 4: Defining RF Scores as Segments**
* **Step 1:** Define segments for the generated RF scores.
* **Step 2:** Use `seg_map` to convert scores into segments.


In [ ]:
# RFM naming
seg_map = {
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at_Risk',
    r'[1-2]5': 'cant_loose',
    r'3[1-2]': 'about_to_sleep',
    r'33': 'need_attention',
    r'[3-4][4-5]': 'loyal_customers',
    r'41': 'promising',
    r'51': 'new_customers',
    r'[4-5][2-3]': 'potential_loyalists',
    r'5[4-5]': 'champions'
}

rfm['segment'] = rfm['RF_SCORE'].replace(seg_map, regex=True)
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE,RFM_SCORE,segment
master_id,,,,,,,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,10,5.00000,776.07000,5,4,4,54,544,champions
00034aaa-a838-11e9-a2fc-000d3a38a36f,298,3.00000,269.47000,1,2,1,12,121,hibernating
000be838-85df-11ea-a90b-000d3a38a36f,213,4.00000,722.69000,2,3,4,23,234,at_Risk
000c1fe2-a8b7-11ea-8479-000d3a38a36f,27,7.00000,874.16000,5,4,4,54,544,champions
000f5e3e-9dde-11ea-80cd-000d3a38a36f,20,7.00000,1620.33000,5,4,5,54,545,champions


**Task 5: Action Time**

* **Step 1:** Examine the average Recency, Frequency, and Monetary values for each segment.

* **Step 2:** Using the RFM analysis, identify the customers matching the profiles for the two cases below and save their customer IDs as a CSV file.

  * a. FLO is introducing a new women's shoe brand with prices above the general customer preferences. Therefore, they want to specifically contact customers who would be interested in the brand’s promotion and product sales. Customers from the "champions" and "loyal_customers" segments, who have shopped in the women's category, will be contacted. Save the IDs of these customers in a CSV file.

  * b. A discount of about 40% is planned for men's and children's products. Customers who were previously good customers but have not shopped for a long time, those who are dormant, and new customers who are interested in these categories should be specifically targeted. Save the IDs of the suitable customers in a CSV file.


In [ ]:
rfm.groupby("segment").agg({'recency': 'mean', 'frequency': 'mean', 'monetary': 'mean'})

,recency,frequency,monetary
segment,,,
about_to_sleep,114.03165,2.40657,361.64937
at_Risk,242.32900,4.47018,648.32504
cant_loose,235.15913,10.71692,1481.65245
champions,17.14219,8.96510,1410.70894
hibernating,247.42630,2.39147,362.58330
loyal_customers,82.55793,8.35644,1216.25722
need_attention,113.03722,3.73945,553.43664
new_customers,17.97623,2.00000,344.04949
potential_loyalists,36.86974,3.31077,533.74134


In [ ]:
rfm[["segment", "recency", "frequency", "monetary"]].groupby("segment").agg(["mean", "count"])

recency       frequency         monetary      
                         mean count      mean count       mean count
segment                                                             
about_to_sleep      114.03165  1643   2.40657  1643  361.64937  1643
at_Risk             242.32900  3152   4.47018  3152  648.32504  3152
cant_loose          235.15913  1194  10.71692  1194 1481.65245  1194
champions            17.14219  1920   8.96510  1920 1410.70894  1920
hibernating         247.42630  3589   2.39147  3589  362.58330  3589
loyal_customers      82.55793  3375   8.35644  3375 1216.25722  3375
need_attention      113.03722   806   3.73945   806  553.43664   806
new_customers        17.97623   673   2.00000   673  344.04949   673
potential_loyalists  36.86974  2925   3.31077  2925  533.74134  2925
promising            58.69461   668   2.00000   668  334.15334   668

In [ ]:
target_segments = rfm[(rfm["segment"] == "champions") | (rfm["segment"] == "loyal_customers")]
# target_segments = rfm[rfm["segment"].isin(["champions","loyal_customers"])] # It yields the same result as the one above.
target_segments.shape

(5295, 9)

In [ ]:
target_segments.head(10)


,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE,RFM_SCORE,segment
master_id,,,,,,,,,
00016786-2f5a-11ea-bb80-000d3a38a36f,10,5.00000,776.07000,5,4,4,54,544,champions
000c1fe2-a8b7-11ea-8479-000d3a38a36f,27,7.00000,874.16000,5,4,4,54,544,champions
000f5e3e-9dde-11ea-80cd-000d3a38a36f,20,7.00000,1620.33000,5,4,5,54,545,champions
003903e0-abce-11e9-a2fc-000d3a38a36f,68,5.00000,577.47000,4,4,3,44,443,loyal_customers
004bec18-9f7a-11e9-a2fc-000d3a38a36f,99,12.00000,684.91000,3,5,4,35,354,loyal_customers
004d5204-2037-11ea-87bf-000d3a38a36f,27,8.00000,1170.76000,5,5,5,55,555,champions
00534fe4-a6b4-11e9-a2fc-000d3a38a36f,63,16.00000,800.59000,4,5,4,45,454,loyal_customers
006633b6-a6b9-11e9-a2fc-000d3a38a36f,61,19.00000,2050.50000,4,5,5,45,455,loyal_customers
0067708e-1ec6-11ea-aa32-000d3a38a36f,7,5.00000,397.30000,5,4,2,54,542,champions


In [ ]:
print(target_segments.index)
case_1 = pd.DataFrame()
case_1["customer_id"] = target_segments.index
case_1.head()

Index(['00016786-2f5a-11ea-bb80-000d3a38a36f',
       '000c1fe2-a8b7-11ea-8479-000d3a38a36f',
       '000f5e3e-9dde-11ea-80cd-000d3a38a36f',
       '003903e0-abce-11e9-a2fc-000d3a38a36f',
       '004bec18-9f7a-11e9-a2fc-000d3a38a36f',
       '004d5204-2037-11ea-87bf-000d3a38a36f',
       '00534fe4-a6b4-11e9-a2fc-000d3a38a36f',
       '006633b6-a6b9-11e9-a2fc-000d3a38a36f',
       '0067708e-1ec6-11ea-aa32-000d3a38a36f',
       '00736820-a834-11e9-a2fc-000d3a38a36f',
       ...
       'ffa4683a-2007-11ea-87bf-000d3a38a36f',
       'ffa8ed4e-a7a1-11e9-a2fc-000d3a38a36f',
       'ffbbd2fe-ad3e-11e9-a2fc-000d3a38a36f',
       'ffcfc4d8-a1bf-11e9-a2fc-000d3a38a36f',
       'ffd73136-b098-11e9-9757-000d3a38a36f',
       'ffdd5eb8-aee9-11e9-a2fc-000d3a38a36f',
       'ffe0ab60-6c79-11ea-9ddf-000d3a38a36f',
       'ffe5c7e8-ee12-11e9-9346-000d3a38a36f',
       'ffefdd18-77ca-11ea-9b70-000d3a38a36f',
       'fffacecc-ddc3-11e9-a848-000d3a38a36f'],
      dtype='object', name='master_id', length=5

,customer_id
0,00016786-2f5a-11ea-bb80-000d3a38a36f
1,000c1fe2-a8b7-11ea-8479-000d3a38a36f
2,000f5e3e-9dde-11ea-80cd-000d3a38a36f
3,003903e0-abce-11e9-a2fc-000d3a38a36f
4,004bec18-9f7a-11e9-a2fc-000d3a38a36f


In [ ]:
df[df.master_id.isin(target_segments.index) & df.interested_in_categories_12.str.contains("KADIN")]

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,sum_of_orders,sum_of_evers
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.00000,1.00000,139.99000,799.38000,[KADIN],5.00000,939.37000
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.00000,2.00000,159.97000,1853.58000,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",21.00000,2013.55000
8,cfbda69e-5b4f-11ea-aca7-000d3a38a36f,Android App,Android App,2020-01-23,2021-03-07,2021-03-07,2020-01-25,3.00000,2.00000,120.48000,934.21000,"[ERKEK, COCUK, KADIN]",5.00000,1054.69000
11,c2e15af2-9eed-11e9-9897-000d3a38a36f,Android App,Android App,2015-07-05,2021-05-19,2021-05-19,2020-07-10,19.00000,1.00000,90.49000,2771.11000,"[COCUK, KADIN, AKTIFSPOR]",20.00000,2861.60000
33,fb840306-1219-11ea-a001-000d3a38a36f,Ios App,Ios App,2020-01-18,2021-02-05,2021-02-05,2020-01-18,7.00000,1.00000,89.99000,833.65000,"[ERKEK, KADIN, AKTIFSPOR]",8.00000,923.64000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19912,f63aba0e-41d8-11ea-96d9-000d3a38a36f,Android App,Android App,2019-11-17,2021-01-17,2021-01-17,2019-12-14,4.00000,4.00000,661.89000,422.96000,[KADIN],8.00000,1084.85000
19913,81d0da04-a53e-11e9-a2fc-000d3a38a36f,Android App,Android App,2017-11-27,2021-05-29,2021-05-29,2019-11-22,5.00000,2.00000,309.97000,842.14000,"[KADIN, AKTIFSPOR]",7.00000,1152.11000
19914,825e6b00-ab40-11e9-a2fc-000d3a38a36f,Ios App,Ios App,2017-10-29,2021-04-19,2021-04-19,2021-03-12,4.00000,3.00000,369.96000,599.93000,"[KADIN, AKTIFSPOR]",7.00000,969.89000
19917,f8c471c8-2596-11eb-81e9-000d3a38a36f,Mobile,Offline,2020-11-20,2021-05-25,2021-03-10,2021-05-25,5.00000,2.00000,134.98000,879.96000,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",7.00000,1014.94000
